# 5.4 Test and Validation Errors

In the previous section, we saw that training error is not a great measure of a model's quality. For example, a $1$-nearest neighbor model will have a training error of $0.0$ (or close to it), but it is not necessarily the best prediction model, especially if there are outliers in the training data.

In order to come up with a better measure of model quality, we need to formalize what it is we want to measure.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
pd.options.display.max_rows = 5

housing_df = pd.read_csv("https://raw.githubusercontent.com/dlsun/data-science-book/master/data/AmesHousing.txt",
                         sep="\t")
housing_df

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2928,2929,924100070,20,RL,77.0,10010,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2006,WD,Normal,170000
2929,2930,924151050,60,RL,74.0,9627,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,11,2006,WD,Normal,188000


# Overfitting and Test Error

Ultimately, the goal of any prediction model is to make predictions on _future_ data. Therein lies the problem with training error. Training error measures how well a model predicts on the current data. It is possible to build a model that **overfits** to the training data---that is, a model that fits so well to the current data that it does poorly on future data.

For example, consider fitting two different models to the 10 training observations shown below. The model represented by the red line actually passes through every observation (that is, its training error is zero). However, most people would prefer the model represented by the blue line. If one had to make a prediction for $y$ when $x = 0.8$, the value of the blue line is intuitively more plausible than the value of the red line, which is out of step with the nearby points.

![](overfitting.png)

The argument for the blue model depends on _future_ data because the blue model is actually worse than the red model on the current data. The red model tries so hard to get the predictions on the training data right that it ends up _overfitting_.

If the goal is to build a model that performs well on future data, then we ought to evaluate it (i.e., by calculating MSE, MAE, etc.) on future data. The prediction error on future data is also known as **test error**, in contrast to training error, which is the prediction error on current data. To calculate the test error, we need _labeled_ future data. In many applications, future data is expensive to collect and _labeled_ future data is even more expensive. How can we approximate the test error, using just the data that we already have?

# Validation Error

The solution is to split the training data into a **training set** and a **validation set**. The model will only be fit on the observations of the training set. Then, the model will be evaluated on the validation set. Because the validation set has not been seen by the model already, it essentially plays the role of "future" data, even though it was carved out of the current data.

The prediction error on the validation set is known as the **validation error**. The validation error is an approximation to the test error.

To split our data into training and validation sets, we use the `.sample()` function in `pandas`. Let's use this to split our data into two equal halves, which we will call `train` and `val`.

In [2]:
train_df = housing_df.sample(frac=.5)
val_df = housing_df.drop(train_df.index)

train_df

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
1662,1663,527402090,80,RL,70.0,10500,Pave,NaN,Reg,Lvl,...,0,NaN,GdWo,NaN,0,12,2007,WD,Normal,139000
2065,2066,905225020,60,RL,99.0,16779,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,Shed,500,5,2007,WD,Normal,158000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,1886,534276180,20,RL,74.0,7450,Pave,NaN,IR1,Lvl,...,0,NaN,GdWo,NaN,0,1,2007,WD,Normal,124000
1758,1759,528290170,60,RL,58.0,9487,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,6,2007,WD,Normal,187000


Now let's use this training/validation split to approximate the test error of a 10-nearest neighbors model. We use Scikit-Learn to preprocess the training and the validation data. Note that the transformer and the scaler are both fit to the training data, so we learn the categories, the mean, and standard deviation from the training set---and use these to transform both the training and validation sets.

In [3]:
# Define features in our model.
features = ["Lot Area", "Gr Liv Area",
            "Full Bath", "Half Bath", "Bedroom AbvGr",
            "Year Built", "Yr Sold",
            "Neighborhood", "Bldg Type"]

from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# dummy encode the categorical features
encoder = make_column_transformer(
    (OneHotEncoder(sparse=False), ["Neighborhood", "Bldg Type"]),
    remainder="passthrough"
)
encoder.fit(housing_df[features])
X_train, y_train = encoder.transform(train_df[features]), train_df["SalePrice"]
X_val, y_val = encoder.transform(val_df[features]), val_df["SalePrice"]

# scale the training features
scaler = StandardScaler()
scaler.fit(X_train)
X_train_sc = scaler.transform(X_train)

# scale the validation features in the same way
X_val_sc = scaler.transform(X_val)

We are now ready to fit a $k$-nearest neighbors model to the training data and make predictions on the validation set.

In [4]:
from sklearn.neighbors import KNeighborsRegressor

# Step 1: Declare the model.
model = KNeighborsRegressor(n_neighbors=10)

# Step 2: Fit the model to the training set.
model.fit(X_train_sc, y_train)

# Step 3: Use the model to predict on the validation set.
y_val_ = model.predict(X_val_sc)

We make predictions on the validation set and calculate the validation RMSE:

In [5]:
rmse = np.sqrt(((y_val - y_val_) ** 2).mean())
rmse

43187.598661363416

Notice that the validation error is higher than the training error that we calculated in the previous section. In general, this will be true. It is harder for a model to predict for new observations it has not seen, than for observations it has seen!

# Cross Validation

One downside of the validation error above is that it was calculated using only 50% of the data. As a result, the estimate is noisy.

There is a cheap way to obtain a second opinion of how well our model will do on future data. Previously, we split our data at random into two halves, training the model on the first half and evaluating it using the second half. Because the model has not already seen the second half of the data, this approximates how well the model would perform on future data. 

But the way we split our data was arbitrary. We might as well swap the roles of the two halves, training the model on the _second_ half and evaluating it using the _first_ half. As long as the model is always evaluated on data that is different from the data that was used to train it, we have a valid measure of how well our model would perform on future data. A schematic of this approach, known as **cross-validation**, is shown below.

<img src="cross-validation.png" />

Because we will be doing all computations twice, just with different data, let's wrap the $k$-nearest neighbors algorithm above into a function called `get_val_error()`, that computes the validation error given training and validation data.

In [6]:
def get_val_error(train_df, val_df):
    
    # convert categorical variables to dummy variables
    encoder = make_column_transformer(
        (OneHotEncoder(sparse=False), ["Neighborhood", "Bldg Type"]),
        remainder="passthrough"
    )
    encoder.fit(housing_df[features])
    X_train, y_train = encoder.transform(train_df[features]), train_df["SalePrice"]
    X_val, y_val = encoder.transform(val_df[features]), val_df["SalePrice"]

    # standardize the training and validation sets
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train_sc = scaler.transform(X_train)
    X_val_sc = scaler.transform(X_val)
    
    # Step 1: Declare the model.
    model = KNeighborsRegressor(n_neighbors=10)

    # Step 2: Fit the model to the training set.
    model.fit(X_train_sc, y_train)

    # Step 3: Use the model to predict on the validation set.
    y_val_ = model.predict(X_val_sc)
    
    rmse = np.sqrt(((y_val - y_val_) ** 2).mean())
    return rmse

If we apply this function to the training and validation sets from earlier, we get the same estimate of the test error.

In [7]:
get_val_error(train_df, val_df)

43187.598661363416

But if we reverse the roles of the training and validation sets, we get another estimate of the test error.

In [8]:
get_val_error(val_df, train_df)

38802.90004412877

Now we have two, somewhat independent estimates of the test error. It is common to average the two to obtain an overall estimate of the test error, called the **cross-validation error**. Notice that the cross-validation error uses each observation in the data exactly once. We make a prediction for each observation, but always using a model that was trained on data that does not include that observation.

## $K$-Fold Cross Validation

Previously, we carried out cross validation by splitting the data into 2 halves, alternately using one half to train the model and the other to evaluate the model. In general, we can split the data into $k$ subsamples, alternately training the data on $k-1$ subsamples and evaluating the model on the $1$ remaining subsample, i.e., the validation set. This produces $k$ somewhat independent estimates of the test error. This procedure is known as **$k$-fold cross validation**. (Be careful not to confuse the $k$ in $k$-fold cross validation with the $k$ in $k$-nearest neighbors.) Therefore, the specific version of cross validation that we saw earlier is $2$-fold cross validation.

A schematic of $4$-fold cross validation is shown below.

![](k-folds.png)

Implementing $k$-fold cross validation from scratch for $k > 2$ is straightforward but messy, so we will usually let Scikit-Learn do it for us.

# Cross Validation in Scikit-Learn

Scikit-Learn provides a function, `cross_val_score`, that will carry out all aspects of $k$-fold cross validation: 

1. split the data into $k$ subsamples
2. combine the first $k-1$ subsamples into a training set and train the model
3. evaluate the model predictions on the last ($k$th) held-out subsample
4. repeat steps 2-3 $k$ times (i.e. $k$ "folds"), each time holding out a different one of the $k$ subsamples
4. calculate $k$ "scores", one from each validation set

There is one subtlety to keep in mind. Training a $k$-nearest neighbors model is not just about fitting the model; it also involves dummifying the categorical variables and scaling the variables. These preprocessing steps should be included in the cross-validation process. They cannot be done ahead of time.

For example, suppose we run $5$-fold cross validation. Then:

- When subsamples 1-4 are used for training and subsample 5 for validation, the observations have to be standardized with respect to the mean and SD of subsamples 1-4.
- When subsamples 2-5 are used for training and subsample 1 for validation, the observations have to be standardized with respect to the mean and SD of subsamples 2-5.
- And so on.

We cannot simply standardize all of the data once at the beginning and run cross validation on the standardized data. To do so would be allowing the model to peek at the validation set during training. That's because each training set would be standardized with respect to a mean and SD that is calculated from all data, including the validation set. To be completely above board, we should standardize each training set with respect to the mean and SD of just that training set.

Fortunately, Scikit-Learn provides a `Pipeline` object that allows us to chain these preprocessing steps together with the model we want to fit.

In [9]:
from sklearn.pipeline import make_pipeline

encoder = make_column_transformer(
    (OneHotEncoder(sparse=False, handle_unknown="ignore"), 
     ["Neighborhood", "Bldg Type"]),
    remainder="passthrough",
    
)
scaler = StandardScaler()
model = KNeighborsRegressor(n_neighbors=10)
pipeline = make_pipeline(encoder, scaler, model)

This entire `Pipeline` can be passed to `cross_val_score`, along with the training data, the number of folds $k$ (`cv`), and the type of score (`scoring`). So $5$-fold cross validation in Scikit-Learn would look as follows:

In [10]:
from sklearn.model_selection import cross_val_score

X, y = housing_df[features], housing_df["SalePrice"]
scores = cross_val_score(pipeline, X, y, 
                         cv=5, scoring="neg_mean_squared_error")
scores

array([-1.93492870e+09, -1.38445796e+09, -1.45629725e+09, -1.78062825e+09,
       -1.38172268e+09])

Notice that we get five (negative) validation MSEs, one from each of the 5 folds. `cross_val_score` returns the _negative_ MSE, instead of the MSE, because by definition, a _higher_ score is better. (Since we want the MSE to be as _low_ as possible, we want the negative MSE to be as _high_ as possible.)

To come up with a single overall estimate of the test MSE, we flip the signs and average the MSEs:

In [11]:
mse = np.mean(-scores)
mse

1587606967.563932

The RMSE is the square root of the MSE:

In [12]:
rmse = np.sqrt(mse)
rmse

39844.7859520406

# Exercises

**Exercise 1.** Use cross-validation to estimate the test error of a 1-nearest neighbor classifier on the Ames housing price data. How does a 1-nearest neighbor classifier compare to a 10-nearest neighbor classifier in terms of its ability to predict on _future_ data?

In [13]:
# YOUR CODE HERE

**Exercise 2.** Using the Tips data set (`https://raw.githubusercontent.com/dlsun/data-science-book/master/data/tips.csv`), train $k$-nearest neighbors regression models to predict the tip for different values of $k$. Calculate the training and validation MAE of each model, and make a plot showing these errors as a function of $k$.

In [14]:
# YOUR CODE HERE